In [1]:
{-# LANGUAGE LambdaCase #-}

import Prelude hiding (read, replicate)
import qualified Prelude as P

import Control.Monad.ST      (ST, runST)
import Data.Array            ((!), bounds)
import Data.Graph            (Graph, buildG, vertices)
import Data.Vector.Mutable   (STVector, read, replicate, write)
import Data.STRef            (STRef, modifySTRef', newSTRef, readSTRef)
import Data.Foldable         (for_)

denormalise     = subtract
normalise       = (+)
other n v       = 2*n - v
clauses n [u,v] = [(other n u, v), (other n v, u)]

whenM :: Monad m => m Bool -> m () -> m ()
whenM boolM block = boolM >>= \b -> if b then block else return ()

tarjan :: Int -> Graph -> [Maybe [Int]]
tarjan n graph = runST $ do
    index    <- newSTRef 0
    stack    <- newSTRef []
    stackSet <- replicate size False
    indices  <- replicate size Nothing
    lowlinks <- replicate size Nothing
    output   <- newSTRef []
    for_ (vertices graph) $ \v ->
        whenM ((Nothing==) <$> read indices v) $
            strongConnect n v graph index stack stackSet indices lowlinks output
    readSTRef output
    where
        size = snd (bounds graph) + 1

strongConnect
    :: Int
    -> Int
    -> Graph
    -> STRef s Int
    -> STRef s [Int]
    -> STVector s Bool
    -> STVector s (Maybe Int)
    -> STVector s (Maybe Int)
    -> STRef s [Maybe [Int]]
    -> ST    s ()
strongConnect n v graph index stack stackSet indices lowlinks output = do
    i <- readSTRef index
    write indices  v (Just i)
    write lowlinks v (Just i)
    increment index
    push stack stackSet v
    for_ (graph ! v) $ \w -> read indices w >>= \case
        Nothing -> do
            strongConnect n w graph index stack stackSet indices lowlinks output
            write lowlinks v =<< min <$> read lowlinks v <*> read lowlinks w
        Just{}  -> whenM (read stackSet w) $
            write lowlinks v =<< min <$> read lowlinks v <*> read indices  w
    whenM ((==) <$> read lowlinks v <*> read indices v) $
        modifySTRef' output =<< (:) <$> addSCC n v (Just []) stack stackSet
    where 
        addSCC :: Int -> Int -> Maybe [Int] -> STRef s [Int] -> STVector s Bool -> ST s (Maybe [Int])
        addSCC n v (Just scc) stack stackSet = do
            w <- pop stack stackSet
            case (other n w) `elem` scc of
                True           -> return Nothing
                False | w == v -> return $ Just (w:scc)
                False          -> addSCC n v (Just (w:scc)) stack stackSet
        
        increment :: STRef s Int -> ST s ()
        increment counter = modifySTRef' counter (+1)

        push :: STRef s [Int] -> STVector s Bool -> Int -> ST s ()
        push stack stackSet e = do
            modifySTRef' stack (e:)
            write stackSet e True

        pop :: STRef s [Int] -> STVector s Bool -> ST s Int
        pop stack stackSet = do
            e <- head <$> readSTRef stack
            modifySTRef' stack tail
            write stackSet e False
            return e

In [2]:
checkSat :: String -> IO Bool
checkSat name = do
    p <- map (map P.read . words) . lines <$> readFile name
    let pNo    = head $ head p
        pn     = map (map (normalise pNo)) $ tail p
        pGraph = buildG (0,2*pNo) $ concatMap (clauses pNo) pn
    return . (Nothing /=) . sequence $ tarjan pNo pGraph

In [3]:
checkSat "2sat1.txt"

True

In [4]:
checkSat "2sat2.txt"

False

In [5]:
checkSat "2sat3.txt"

True

In [6]:
checkSat "2sat4.txt"

True

In [7]:
checkSat "2sat5.txt"

False

In [8]:
checkSat "2sat6.txt"

False